<a href="https://colab.research.google.com/github/fatemafaria142/Math-Word-Problem/blob/main/Math_Word_Problem_Solving_using_openchat_3_5_1210.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00


#### **Dataset Link:** https://huggingface.co/datasets/camel-ai/math

In [16]:
from datasets import load_dataset

# Load the dataset
instruct_tune_dataset = load_dataset("camel-ai/math")

# Filter the dataset to include only instances where the topic is "Algebra" or "Trigonometry"
instruct_tune_dataset['train'] = instruct_tune_dataset['train'].filter(lambda example: example['topic;'] in ['Algebra', 'Trigonometry'])


### **Dataset structure**
* The dataset contains five columns.

In [17]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['role_1', 'topic;', 'sub_topic', 'message_1', 'message_2'],
        num_rows: 4000
    })
})

In [19]:
# Display information for 5 data points from the 'train' split
num_samples_to_show = 5
for i in range(num_samples_to_show):
    data = instruct_tune_dataset['train'][i]
    print(f"Data Point {i + 1}:")
    print("Role:", data['role_1'])
    print("Topic:", data['topic;'])
    print("Sub Topic:", data['sub_topic'])
    print("Message 1:", data['message_1'])
    print("Message 2:", data['message_2'])
    print("\n-----------------------------\n")


Data Point 1:
Role: Mathematician_RoleType.ASSISTANT
Topic: Algebra
Sub Topic: Solving linear equations
Message 1: Solve for x: 5x + 3 = 28
Message 2: To solve for x, we need to isolate x on one side of the equation. 

Step 1: Subtract 3 from both sides of the equation.
5x + 3 - 3 = 28 - 3
5x = 25

Step 2: Divide both sides of the equation by 5.
5x / 5 = 25 / 5
x = 5

So, x = 5.

-----------------------------

Data Point 2:
Role: Mathematician_RoleType.ASSISTANT
Topic: Algebra
Sub Topic: Solving linear equations
Message 1: Solve for x: 2x + 5 = 9- x.
Message 2: To solve for x, we will follow these steps:

1. Add x to both sides of the equation to get all the x terms on one side:
2x + x + 5 = 9 - x + x

This simplifies to:
3x + 5 = 9

2. Subtract 5 from both sides of the equation to isolate the x terms:
3x + 5 - 5 = 9 - 5

This simplifies to:
3x = 4

3. Divide both sides of the equation by 3 to solve for x:
3x / 3 = 4 / 3

This simplifies to:
x = 4/3

So, the solution is x = 4/3.

-----

### **We will use just a small subset of the data for this training example**

In [21]:
instruct_tune_dataset["train"] = instruct_tune_dataset["train"].select(range(4000))
instruct_tune_dataset["test"] = instruct_tune_dataset["train"].select(range(400))

In [22]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['role_1', 'topic;', 'sub_topic', 'message_1', 'message_2'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['role_1', 'topic;', 'sub_topic', 'message_1', 'message_2'],
        num_rows: 400
    })
})

In [23]:
def create_prompt(sample):
    # Use a predefined template for instructions
    prompt = "Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently. "
    prompt += f"My Question is: {sample['message_1']}\n"
    single_turn_prompt = f"Math Correct User: {prompt}<|end_of_turn|>Math Correct Assistant: {sample['message_2']}"
    return single_turn_prompt

In [24]:
create_prompt(instruct_tune_dataset["train"][0])

'Math Correct User: Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently. My Question is: Solve for x: 5x + 3 = 28\n<|end_of_turn|>Math Correct Assistant: To solve for x, we need to isolate x on one side of the equation. \n\nStep 1: Subtract 3 from both sides of the equation.\n5x + 3 - 3 = 28 - 3\n5x = 25\n\nStep 2: Divide both sides of the equation by 5.\n5x / 5 = 25 / 5\nx = 5\n\nSo, x = 5.'

In [25]:
create_prompt(instruct_tune_dataset["train"][1])

'Math Correct User: Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently. My Question is: Solve for x: 2x + 5 = 9- x.\n<|end_of_turn|>Math Correct Assistant: To solve for x, we will follow these steps:\n\n1. Add x to both sides of the equation to get all the x terms on one side:\n2x + x + 5 = 9 - x + x\n\nThis simplifies to:\n3x + 5 = 9\n\n2. Subtract 5 from both sides of the equation to isolate the x terms:\n3x + 5 - 5 = 9 - 5\n\nThis simplifies to:\n3x = 4\n\n3. Divide both sides of the equation by 3 to solve for x:\n3x / 3 = 4 / 3\n\nThis simplifies to:\nx = 4/3\n\nSo, the solution is x = 4/3.'

In [26]:
create_prompt(instruct_tune_dataset["train"][2])

'Math Correct User: Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently. My Question is: Solve the equation: 3x + 5 = 14.\n<|end_of_turn|>Math Correct Assistant: To solve the equation 3x + 5 = 14, we need to isolate the variable x. \n\nStep 1: Subtract 5 from both sides of the equation.\n3x + 5 - 5 = 14 - 5\n3x = 9\n\nStep 2: Divide both sides of the equation by 3.\n3x / 3 = 9 / 3\nx = 3\n\nSo, the solution to the equation 3x + 5 = 14 is x = 3.'

In [27]:
create_prompt(instruct_tune_dataset["train"][3])

'Math Correct User: Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently. My Question is: Solve for x: 2x + 5 = 17.\n<|end_of_turn|>Math Correct Assistant: To solve for x, we need to isolate x on one side of the equation. We can do this by following these steps:\n\n1. Subtract 5 from both sides of the equation:\n   2x + 5 - 5 = 17 - 5\n   2x = 12\n\n2. Divide both sides of the equation by 2:\n   2x / 2 = 12 / 2\n   x = 6\n\nSo, the solution is x = 6.'

In [28]:
create_prompt(instruct_tune_dataset["train"][4])

'Math Correct User: Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently. My Question is: Solve the equation 4x - 5 = 11.\n<|end_of_turn|>Math Correct Assistant: To solve the equation 4x - 5 = 11, follow these steps:\n\nStep 1: Add 5 to both sides of the equation.\n4x - 5 + 5 = 11 + 5\n\nThis simplifies to:\n4x = 16\n\nStep 2: Divide both sides of the equation by 4.\n4x / 4 = 16 / 4\n\nThis simplifies to:\nx = 4\n\nSo the solution to the equation 4x - 5 = 11 is x = 4.'

In [29]:
create_prompt(instruct_tune_dataset["train"][5])

'Math Correct User: Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently. My Question is: Solve for x: 4x + 5 = 17.\n<|end_of_turn|>Math Correct Assistant: To solve for x, we need to isolate x on one side of the equation. We can do this by following these steps:\n\n1. Subtract 5 from both sides of the equation:\n   4x + 5 - 5 = 17 - 5\n   4x = 12\n\n2. Divide both sides of the equation by 4:\n   4x / 4 = 12 / 4\n   x = 3\n\nSo, the solution is x = 3.'

### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [30]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )


* **Model and Tokenizer:** https://huggingface.co/openchat/openchat-3.5-1210

In [31]:
mode_id = "openchat/openchat-3.5-1210"

In [32]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [33]:
tokenizer = AutoTokenizer.from_pretrained("openchat/openchat-3.5-1210")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### **Let's example how well the model does at this task currently:**

In [34]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=256, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "")

In [38]:
prompt = "Math Correct User: Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently. "
prompt += "My Question is: Solve the equation: 3x + 5 = 14. <|end_of_turn|>"
prompt += "Math Correct Assistant:"

In [39]:
print(prompt)

Math Correct User: Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently. My Question is: Solve the equation: 3x + 5 = 14. <|end_of_turn|>Math Correct Assistant:


In [40]:
generate_response(prompt, model)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


'<s> Math Correct User: Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently. My Question is: Solve the equation: 3x + 5 = 14. <|end_of_turn|> Math Correct Assistant: To solve the equation 3x + 5 = 14, we want to isolate the variable x.\nFirst, we subtract 5 from both sides of the equation:\n3x + 5 - 5 = 14 - 5\nThis simplifies to:\n3x = 9\nNext, we divide both sides of the equation by 3:\n3x / 3 = 9 / 3\nThis simplifies to:\nx = 3\nTherefore, the solution to the equation 3x + 5 = 14 is x = 3.\n#### 3\nThe answer is: 3<|end_of_turn|>'

### **Setting up the Training**
we will be using the `huggingface` and the `peft` library!

In [41]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")


* we need to prepare the model to be trained in 4bit so we will use the  **`prepare_model_for_kbit_training`** function from peft




In [42]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# **Training Hyperparameters**
The choice of hyperparameters is contingent upon the desired training duration. Pay special attention to the following key factors:

* `num_train_epochs/max_steps:` Dictates the number of iterations over the data. Exercise caution, as an excessive number may lead to overfitting!

* `learning_rate:` Governs the convergence speed of the model. Adjust this parameter judiciously for optimal results.

In [43]:
from transformers import TrainingArguments
output_model= "openchat_instruct_generation"
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=250,
        fp16=True,
)


### **Setting up the trainer**

`max_seq_length`: Context window size


In [44]:
from trl import SFTTrainer

max_seq_length = 256

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt, # this will aplly the create_prompt mapping to all training and test dataset
  args=training_arguments,
  train_dataset=instruct_tune_dataset["train"],
  eval_dataset=instruct_tune_dataset["test"]
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:310: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


### **Training starts here**

In [45]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.167900
20,0.757200
30,0.556400
40,0.447300
50,0.459700
60,0.498700
70,0.431500
80,0.416600
90,0.378800
100,0.344400


TrainOutput(global_step=250, training_loss=0.45344038772583006, metrics={'train_runtime': 1354.2748, 'train_samples_per_second': 0.738, 'train_steps_per_second': 0.185, 'total_flos': 1.092722098176e+16, 'train_loss': 0.45344038772583006, 'epoch': 0.16})

### **Save the model**

In [46]:
trainer.save_model("openchat_instruct_generation")

In [47]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:229: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [48]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=256, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0]

### **Example No:1**

In [50]:
prompt = "Math Correct User: Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently. "
prompt += "My Question is: Solve the equation: 3x + 5 = 14. <|end_of_turn|>\n"
prompt += "Math Correct Assistant:"
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)

<s> Math Correct User: Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently. My Question is: Solve the equation: 3x + 5 = 14. <|end_of_turn|> 
Math Correct Assistant: To solve the equation 3x + 5 = 14 for x, follow these steps:

Step 1: Subtract 5 from both sides of the equation.
3x + 5 - 5 = 14 - 5
3x = 9

Step 2: Divide both sides of the equation by 3.
3x / 3 = 9 / 3
x = 3

So, the solution is x = 3.<|end_of_turn|>


### **Example No:2**

In [56]:
prompt = "Math Correct User: Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently. "
prompt += "My Question is: Solve the inequality 2x + 5 > 11, and express the solution set in interval notation. <|end_of_turn|>\n"
prompt += "Math Correct Assistant:"
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)

<s> Math Correct User: Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently. My Question is: Solve the inequality 2x + 5 > 11, and express the solution set in interval notation. <|end_of_turn|> 
Math Correct Assistant: To solve the inequality 2x + 5 > 11, we need to isolate x. We can do this by following these steps:

Step 1: Subtract 5 from both sides of the inequality.
2x + 5 - 5 > 11 - 5
2x > 6

Step 2: Divide both sides of the inequality by 2.
2x/2 > 6/2
x > 3

So the solution to the inequality is x > 3. In interval notation, this can be written as (3, ∞).<|end_of_turn|>


### **Example No:3**

In [57]:
prompt = "Math Correct User: Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently. "
prompt += "My Question is: Solve the following rational equation for x: \n\n(2x-1)/(x+3) = 5/2 <|end_of_turn|>\n"
prompt += "Math Correct Assistant:"
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)

<s> Math Correct User: Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently. My Question is: Solve the following rational equation for x: 

(2x-1)/(x+3) = 5/2 <|end_of_turn|> 
Math Correct Assistant: To solve the rational equation (2x-1)/(x+3) = 5/2, we can start by cross-multiplying to get rid of the fractions.

(2x-1)/(x+3) * (2/(5)) = (5/2) * (2/(5))

Simplifying, we get:

(4x-2) / (x+3) * (2/5) = 1

Now, we can get rid of the fraction by multiplying both sides by the reciprocal of the left side, which is (x+3) * (5/2).

(4x-2) * (2/5) = (x+3) * (5/2) * (2/5)

Simplifying again, we get:

(8x-4) = 5(x+3)

Now, distribute the 5 on the right side:

8x - 4 = 5x + 15

Next, subtract 5x from both sides:

8x - 5x - 4 = 5x - 


### **Example No:4**

In [58]:
prompt = "Math Correct User: Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently. "
prompt += "My Question is: Simplify the expression: 2^3 × 2^4 ÷ 2^2 <|end_of_turn|>\n"
prompt += "Math Correct Assistant:"
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)

<s> Math Correct User: Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently. My Question is: Simplify the expression: 2^3 × 2^4 ÷ 2^2 <|end_of_turn|> 
Math Correct Assistant: To simplify the expression, we use the properties of exponents. 

Given expression: 2^3 × 2^4 ÷ 2^2

Using the property of exponents, a^m × a^n = a^(m+n), we can combine the exponents:

2^(3+4) ÷ 2^2

2^7 ÷ 2^2

Now, using the property of exponents, a^m ÷ a^n = a^(m-n), we can simplify further:

2^(7-2)

2^5

So, the simplified expression is 2^5, which is equal to 32.<|end_of_turn|>
